# Predicting Peak Player Activity: Optimizing Game Server Resources

## Group Members: Sungjun Lee, Prima Cheung, Sayra Arij, Mishaa Murugathass

## Introduction

Predicting player activity patterns is crucial for optimizing server resource allocation, reducing latency, and improving overall gameplay satisfaction. Studies have shown that load prediction services can anticipate future game world entity distributions from historical data, enabling more efficient resource provisioning. Additionally, analyzing player behavior and gaming patterns allows for smarter allocation of resources, as metrics such as peak usage times and session lengths play a crucial role in this process.

This study aims to predict peak hours of player activity using historical session data from a video game research server. The dataset used in this project consists of two files: sessions.csv and players.csv. The focus of this report is on sessions.csv, which records play sessions from a video game research server. Key attributes include unique player identifiers, session start and end times, and Unix timestamp formats. However, potential issues such as missing end_time values, timestamp granularity, and data collection bias may affect analysis accuracy. The players.csv file provides additional information about the players themselves. It comprises 196 rows and 7 columns, including variables such as player experience, subscription status, total play hours, and demographic information.

The question we are asking is: **Can we predict the peak hours of player activity using historical session data?** Understanding peak play hours enables server administrators to allocate resources efficiently, ensuring smooth gameplay experiences without overloading or underutilizing server capacity. By leveraging historical session data, this study aims to develop a predictive model to inform decision-making in server management and resource distribution.



## Methods & Results

#### Load and Prepare Data

In [ ]:
library(tidyverse)

sessions <- read_csv("https://raw.githubusercontent.com/anderpudding/dsci-project-005-16/refs/heads/main/data/sessions.csv")

#### Wrangle and Tidy Data
We were able to select the neccessary components of the data tables and make it into a tidy data table. The columns neccessary for the question are the start and stop times of each session, which allows us to calculate for the duration of each session.

In [ ]:
sessions <- sessions |>
    mutate(start_time = strptime(start_time, "%d/%m/%Y %H:%M"),
        end_time = strptime(end_time, "%d/%m/%Y %H:%M"),
        session_duration = as.numeric(difftime(end_time, start_time, units = "mins")))

sessions

#### Summary Statistics
This step allowed us to summarize the data needed. Since we are trying to determine the duration of played sessions, the best way to summarize this data is by determining the mean session duration. The mean number of player per hour was also determined in the same way.

In [ ]:
summary_stats <- sessions |>
    summarise(mean_session_duration = mean(session_duration, na.rm = TRUE),
        mean_players_per_hour = n() / length(unique(format(start_time, "%H"))))

summary_stats

#### Extract Time Features
We then needed to find the time stamps of which there are the most number of players active. This was done by grouping the start times of each session. These session times were recorded by the hour of the day and day ofhte week.

In [ ]:
sessions <- sessions |>
    mutate(hour = as.numeric(format(start_time, "%H")),
        day_of_week = weekdays(as.Date(start_time)))

#### Visualization 1: Player Activity by Hour
The below plot tells us how many players started at certain times of the day. This was done by grouping each recorded session by their start time of each day (which is the "hour" column made above) and determining the number of players who logged on at that time. This step was done using the group_by + summairze functions. The created dataset was then made into a line plot using ggplot. A line plot was chosen for this dataset because the number of players on the game is continuous over the suggested time range.

In [ ]:
hourly_counts <- sessions |> 
    group_by(hour) |> 
    summarise(player_count = n())

ggplot(hourly_counts, aes(x = hour, y = player_count)) +
    geom_line() +
    geom_point() +
    labs(title = "Figure 1: Player Activity by Hour", x = "Hour of Day", y = "Number of Sessions")

#### Visualization 2: Player Activity by Day of Week
After determining the number of players by the hour, it is also essential to consider how the day of the week will affect the number of players active. Using a similar group_by + summarize approach, the number of players for each day of the week can be determined. Using ggplot, a bar plot was made to show the player activity by days of the week. A bar plot was chosen for this data analysis because player activity is not a continuous variable over each day of the week (the number of players for each day of the week is independent of each other).

In [ ]:
daily_counts <- sessions |> 
    group_by(day_of_week) |> 
    summarise(player_count = n())

ggplot(daily_counts, aes(x = day_of_week, y = player_count, fill = day_of_week)) +
    geom_bar(stat = "identity") +
    labs(title = "Figure 2: Player Activity by Day of Week", x = "Day", y = "Number of Sessions", fill = "Day of the Week")

## Discussion

From the analysis, we have found meaningful trends to help our study predict the peak hours. From the historical data, it reveals that the player's activity rises significantly after 1AM and then drops substantially before 5AM. From figure 1, it is evident that the lowest number of sessions happens in the afternoon and then slowly increases again after 2PM. Given that between those hours, most people are either busy with school or work. This pattern is further proved with figure 2, due to the number of sessions being the highest on Saturdays, followed by Sundays. 

These findings were expected as it follows the periods of time where people are free and able to do their leisure activities. It was more expected to see the peak hours of the number of sessions to be during the evenings. Therefore, it was a little surprising for the number of sessions to be highest around midnight. Although it confirmed our predictions on the weekends to have the highest number of sessions. 

This informs us that weekends and times around midnight is where we need to ensure that the number of licenses on hand is sufficiently large to accommodate that nearly all players can connect and play without disruption. By focusing on past data, we now have more information to have more smoother games during the busiest times. The implications of these findings can further help us to optimize resource allocation, reduce server lag, and improve the overall player experience by being better prepared for peak demand.

Future questions that may arise from our found data may be: How does specific player demographics (e.g., experience level or subscription status) further influence peak times? What role does social behavior (e.g., playing with friends or in teams) play in peak times? By looking at additional reasons that may affect peak hours is a route that may be helpful to better understand player habits.